In [1]:
# 0) install BERTScore
!pip --quiet install bert-score
!pip --quiet install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 49.9 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import time
import pandas as pd
import torch
from PIL import Image, ImageOps
from tqdm.auto import tqdm
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from peft import PeftModel
from transformers import AutoTokenizer
from bert_score import score

2025-05-17 18:50:45.869536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747507846.072038      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747507846.125734      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 1) loading QA dataset
df = pd.read_csv("/kaggle/input/test-data-curation/qa_dataset.csv")

In [4]:
# 2) device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# 3) load Qwen2‐VL‐2B‐Instruct baseline
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)


# 2) Plug in your LoRA adapter and merge for inference
model_ft = PeftModel.from_pretrained(
    model,
    "/kaggle/input/16-may-qwen-2-finetuned/lora_adapters_fp16",  
    torch_dtype=torch.float16
)
model_ft = model_ft.merge_and_unload()  # fuse adapter into base
model_ft.eval()

# 3) Reload your processor (same as baseline)
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    use_fast=True,
    trust_remote_code=True
)



config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [6]:
class PadTo256:
    def __init__(self, fill=255):
        # fill=255 gives white; for RGB you could also do fill=(255,255,255)
        self.fill = fill

    def __call__(self, img: Image.Image) -> Image.Image:
        # 1) make a copy so we don't clobber the original
        img = img.copy()

        # 2) thumbnail will scale **in-place** so that both width & height <= (256,256)
        #    preserving aspect ratio
        img.thumbnail((256, 256), Image.BILINEAR)

        w, h = img.size
        # 3) compute how much padding is needed on each side
        pad_left   = (256 - w) // 2
        pad_top    = (256 - h) // 2
        pad_right  = 256 - w - pad_left
        pad_bottom = 256 - h - pad_top

        # 4) expand with white border
        return ImageOps.expand(img, border=(pad_left, pad_top, pad_right, pad_bottom),
                               fill=self.fill)


# instantiate once:
RESIZE = PadTo256(fill=(255,255,255))   # white pad
# in collate_fn, just:

In [8]:
import math

# 1) parameters
batch_size  = 8
preds, refs = [], []
image_root  = "/kaggle/input/test-data-dataset/images"
n_samples   = len(df)
n_batches   = math.ceil(n_samples / batch_size)

# 2) batched inference
start_time = time.time()
for batch_start in tqdm(range(0, n_samples, batch_size),
                        total=n_batches,
                        desc="Qwen2-VL Batched Inference"):
    batch_df = df.iloc[batch_start : batch_start + batch_size]

    # 2a) build list of “chat” messages for the batch
    batch_messages = []
    for _, row in batch_df.iterrows():
        img = Image.open(os.path.join(image_root, row["path"])).convert("RGB")
        img = RESIZE(img)
        batch_messages.append([
            {"role": "user", "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": f"Answer in exactly one word: {row['question']}"}
            ]}
        ])

    # 2b) apply the chat template in batch
    batch_texts = [
        processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
        for msg in batch_messages
    ]

    # 2c) extract vision inputs for the whole batch
    image_inputs, video_inputs = process_vision_info(batch_messages)

    # 2d) tokenize the entire batch at once
    inputs = processor(
        text=batch_texts,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to(device)

    # 2e) generate in one shot
    with torch.inference_mode():
        generated = model_ft.generate(**inputs, max_new_tokens=32)

    # 2f) decode each example, trimming off the prompt
    for in_ids, out_ids in zip(inputs.input_ids, generated):
        trimmed_ids = out_ids[len(in_ids):]
        text_out   = processor.batch_decode(
            [trimmed_ids],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0].strip()
        preds.append(text_out)

    # 2g) collect references
    refs.extend(batch_df["answer"].astype(str).str.strip().tolist())

end_time = time.time()

Qwen2-VL Batched Inference:   0%|          | 0/750 [00:00<?, ?it/s]

In [9]:
# # 4) inference loop with timing
# preds, refs = [], []
# image_root  = "/kaggle/input/test-data-dataset/images"

# start_time = time.time()
# for _, row in tqdm(df.iterrows(), total=len(df), desc="Qwen2-VL Inference"):
#     # load & preprocess image
#     img_path = os.path.join(image_root, row["path"])
#     image    = Image.open(img_path).convert("RGB").resize((256, 256))
    
#     # build the chat‐style message
#     messages = [
#         {
#             "role": "user",
#             "content": [
#                 {"type": "image", "image": image},
#                 {"type": "text",  "text":  row["question"]}
#             ],
#         }
#     ]
#     # apply template
#     text = processor.apply_chat_template(
#         messages, tokenize=False, add_generation_prompt=True
#     )
#     # extract vision inputs
#     image_inputs, video_inputs = process_vision_info(messages)
#     # tokenize & move to device
#     inputs = processor(
#         text=[text],
#         images=image_inputs,
#         videos=video_inputs,
#         padding=True,
#         return_tensors="pt",
#     ).to(device)

#     # generate
#     with torch.inference_mode():
#         generated_ids = model.generate(**inputs, max_new_tokens=32)

#     # trim off prompt tokens
#     trimmed_ids = [
#         out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
#     ]
#     output_text = processor.batch_decode(
#         trimmed_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
#     )
#     pred = output_text[0].strip()

#     preds.append(pred)
#     refs.append(str(row["answer"]).strip())

# end_time = time.time()

In [10]:
# 5) compute metrics
total_time    = end_time - start_time
avg_time_ms   = total_time / len(preds) * 1000

# Exact‐Match Accuracy
exact_match = sum(p.lower()==r.lower() for p,r in zip(preds, refs)) / len(preds)

# BERTScore F1 (all preds)
P_all, R_all, F1_all = score(preds, refs, lang="en", rescale_with_baseline=True)
bert_f1_all = F1_all.mean().item()

# One‐word analysis
is_one_word  = [((" " not in p) and p!= "") for p in preds]
pct_one_word = sum(is_one_word) / len(preds) * 100
preds_1w     = [p for p, ok in zip(preds, is_one_word) if ok]
refs_1w      = [r for r, ok in zip(refs,  is_one_word) if ok]

if preds_1w:
    _, _, F1_1w = score(preds_1w, refs_1w, lang="en", rescale_with_baseline=True)
    bert_f1_1w = F1_1w.mean().item()
else:
    bert_f1_1w = float("nan")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 6) report
print(f"Total inference time (1000 samples): {total_time:.1f}s")
print(f"Average per sample:                {avg_time_ms:.1f}ms\n")
print(f"Exact-Match Accuracy:              {exact_match:.2f}")
print(f"BERTScore F1 (all preds):          {bert_f1_all:.2f}\n")
print(f"% One-Word Predictions:            {pct_one_word:.2f}%")
print(f"BERTScore F1 (one-word only):      {bert_f1_1w:.2f}")

Total inference time (1000 samples): 619.2s
Average per sample:                103.3ms

Exact-Match Accuracy:              0.66
BERTScore F1 (all preds):          0.84

% One-Word Predictions:            96.28%
BERTScore F1 (one-word only):      0.87
